## The Tools

In [40]:
#navigation
import os

#data wrangling
import pandas as pd
import numpy as np

#model building
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import cross_val_predict, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from sklearn.feature_selection import RFE
from sklearn.metrics import log_loss

#model scoring
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, r2_score, mean_squared_error, \
confusion_matrix, precision_score, f1_score, matthews_corrcoef

#data visualization
import matplotlib.pyplot as plt
import pylab 

## Helper Functions

In [37]:
def diagnose_results(dataframe, actual_values_col='stabilityscore_2classes',predicted_values_col='stabilityscore_2classes_predictions',pred_prob='stabilityscore_2classes_prob_predictions'):
    """
    The dataframe must be a dataframeframe that includes the predicted results
    
    """
    true = dataframe[dataframe[predicted_values_col]==dataframe[actual_values_col]]
    false = dataframe[dataframe[predicted_values_col]!=dataframe[actual_values_col]]

    tn = true[true[predicted_values_col]==False]
    tp = true[true[predicted_values_col]==True]
    fp = false[false[predicted_values_col]==True]
    fn = false[false[predicted_values_col]==False]

    
    total = len(true)+len(false)
    accuracy = len(true)/total
    # RESULTS FOR CUSTOM RUN
    
    print("\tUNSPECIFIED TOPOLOGY RESULTS:\n")

    print("TP count: %s\nTN count: %s\nFP count: %s\nFN count: %s"%(len(tp),len(tn),len(fp),len(fn)))
    print("Accuracy: %.4f\n" %accuracy)
    print("Log loss: %s\n"%log_loss(dataframe[actual_values_col],dataframe[pred_prob],normalize=False))
    
    #most sure the prediction was in the times it was correct that it was stable
    print("most sure the prediction was in the times it was correct that it was stable: %0.2f \n\tavg. value: %0.2f"%(tp[pred_prob].max(),tp[pred_prob].mean()))


    #most sure the prediction was in the times it was correct that it was unstable
    print("most sure the prediction was in the times it was correct that it was unstable: %0.2f \n\tavg. value: %0.2f"%(tp[pred_prob].max(),tn[pred_prob].mean()))
          

    #most sure the prediction was in the times it was incorrect about it being stable
    print("most sure the prediction was in the times it was incorrect about it being stable: %0.2f \n\tavg. value: %0.2f"%(tp[pred_prob].max(),fp[pred_prob].mean()))
          

    #most sure prediction was in times it was incorrect about it being unstable
    print("most sure the prediction was in the times it was incorrect about it being unstable: %0.2f \n\tavg. value: %0.2f"%(tp[pred_prob].max(),fn[pred_prob].mean()))
    
    
    print("")

    topologies = dataframe['topology'].unique()
    for topology in topologies:
        print("\t%s TOPOLOGY RESULTS:"%topology.upper())

        print("TP count: %s\nTN count: %s\nFP count: %s\nFN count: %s\n"%(len(tp[tp['topology']==topology]),len(tn[tn['topology']==topology]),
                                                                            len(fp[fp['topology']==topology]),len(fn[fn['topology']==topology])))


        #most sure the prediction was in the times it was correct that it was stable
        print("most sure the prediction was in the times it was correct that it was stable:\n",
              tp[tp['topology']==topology][pred_prob].max())


        #most sure the prediction was in the times it was correct that it was unstable
        print("most sure the prediction was in the times it was correct that it was unstable:\n",
              tn[tn['topology']==topology][pred_prob].max())

        #most sure the prediction was in the times it was incorrect about it being stable
        print("most sure the prediction was in the times it was incorrect about it being stable:\n",
              fp[fp['topology']==topology][pred_prob].max())

        #most sure prediction was in times it was incorrect about it being unstable
        print("most sure the prediction was in the times it was incorrect about it being unstable:\n",
              fn[fn['topology']==topology][pred_prob].max())

        print('')


## Random Forest Analysis (Hamed Classifier)

In [38]:
#Load data from run_ENLq65pLA62pJ and leave one out(loo) data
custom_run_data = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_5dx1oGN6A25pw/run_ENLq65pLA62pJ/testing_data.csv')
loo_run_data_heeh = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_5dx1oGN6A25pw/loo_5rl3Ya7ZNp1qr/run_6aoNdQ5wB99gJ/testing_data.csv')
loo_run_data_ehee = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_5dx1oGN6A25pw/loo_5rl3Ya7ZNp1qr/run_E6YBdmdv6E78e/testing_data.csv')
loo_run_data_eehee = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_5dx1oGN6A25pw/loo_5rl3Ya7ZNp1qr/run_QyEALbrQvbQND/testing_data.csv')
loo_run_data_hhh = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_5dx1oGN6A25pw/loo_5rl3Ya7ZNp1qr/run_amqkbkrOY1v5k/testing_data.csv')

In [39]:
list1 = [custom_run_data,loo_run_data_eehee,loo_run_data_ehee,loo_run_data_heeh,loo_run_data_hhh]
list2 = ["custom_run_data","loo_run_data_eehee","loo_run_data_ehee","loo_run_data_heeh","loo_run_data_hhh"]

for i in range(len(list1)):
    print('\t',list2[i].upper())
    diagnose_results(list1[i])

	 CUSTOM_RUN_DATA
	UNSPECIFIED TOPOLOGY RESULTS:

TP count: 494
TN count: 2404
FP count: 232
FN count: 102
Accuracy: 0.8967

Log loss: 836.7204253652936

most sure the prediction was in the times it was correct that it was stable: 0.99 
	avg. value: 0.84
most sure the prediction was in the times it was correct that it was unstable: 0.99 
	avg. value: 0.09
most sure the prediction was in the times it was incorrect about it being stable: 0.99 
	avg. value: 0.71
most sure the prediction was in the times it was incorrect about it being unstable: 0.99 
	avg. value: 0.26

	EEHEE TOPOLOGY RESULTS:
TP count: 140
TN count: 801
FP count: 77
FN count: 32

most sure the prediction was in the times it was correct that it was stable:
 0.9861454727403921
most sure the prediction was in the times it was correct that it was unstable:
 0.4985253821198984
most sure the prediction was in the times it was incorrect about it being stable:
 0.9360817989911426
most sure the prediction was in the times it was 

## Random Forest Analysis (Estrada Classifier)

* note: using much less data that Hamed. This is mainly just practice

In [28]:
#Load data from run_ENLq65pLA62pJ
custom_run_data = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_EzYZapqDJ7dDY/run_6lWoN3GzjWkXw/testing_data.csv').drop("Unnamed: 0",axis=1)
loo_run_data_heeh = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_EzYZapqDJ7dDY/loo_6lPjJZoaGryZ3/run_E6Qr5YO8D3wAy/testing_data.csv').drop("Unnamed: 0",axis=1)
loo_run_data_ehee = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_EzYZapqDJ7dDY/loo_6lPjJZoaGryZ3/run_6lBm8o5xDPwkJ/testing_data.csv').drop("Unnamed: 0",axis=1)
loo_run_data_eehee = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_EzYZapqDJ7dDY/loo_6lPjJZoaGryZ3/run_Q9AXDJbOOygz7/testing_data.csv').drop("Unnamed: 0",axis=1)
loo_run_data_hhh = pd.read_csv('/home/jupyter/tacc-work/test-harness-v3/protein-design/test_harness/results/executions/exec_EzYZapqDJ7dDY/loo_6lPjJZoaGryZ3/run_Q95exY1638el7/testing_data.csv').drop("Unnamed: 0",axis=1)

In [29]:
custom_run_data

,dataset,name,S_PC,Mean_H_entropy,Mean_L_entropy,Mean_E_entropy,Mean_res_entropy,SumH_entropies,SumL_entropies,SumE_entropies,...,SS10,SS11,SS12,SS13,SS14,stabilityscore,topology,dataset_original,stabilityscore_predictions,stabilityscore_prob_predictions
0,Rocklin,EEHEE_rd1_1097,0.163910,0.821937,-0.123385,-0.506895,0.196293,0.565217,-0.251196,-0.230439,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,EEHEE,Rocklin,True,0.664006
1,Rocklin,EEHEE_rd4_0736,1.005983,1.860709,-0.151571,-0.393983,0.893820,1.476194,0.326032,-0.223373,...,-0.026725,-0.025017,-0.023439,-0.021986,-0.020647,True,EEHEE,Rocklin,True,0.717613
2,Rocklin,HEEH_rd4_0121,-0.229727,-0.132803,-0.395473,0.154735,-0.278542,-0.020589,-0.235348,-0.242687,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,HEEH,Rocklin,False,0.330126
3,Rocklin,HEEH_rd3_1201,-0.574548,0.367734,-1.896362,0.566658,-0.605713,0.734933,-2.097472,-0.480110,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,HEEH,Rocklin,False,0.109062
4,Rocklin,EEHEE_rd2_0831,0.538165,0.302488,-0.564894,0.493354,0.569686,0.161606,-0.153061,0.965155,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,EEHEE,Rocklin,False,0.273920
5,Rocklin,EEHEE_rd2_0214,0.316101,-1.257687,-0.546610,0.859087,0.347366,-0.668791,0.132201,1.402315,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,EEHEE,Rocklin,False,0.061092
6,Rocklin,HHH_rd1_0714,-0.964835,-0.352015,0.200052,-1.356988,-0.977188,-0.333994,-0.274967,-1.246553,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,HHH,Rocklin,True,0.787931
7,Rocklin,HEEH_rd3_0717,0.083029,-0.224578,0.162700,1.080675,0.018184,-0.122205,0.357728,0.048439,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,HEEH,Rocklin,False,0.035458
8,Rocklin,HEEH_rd1_0579,-1.046674,-0.519452,-0.780916,-0.353193,-1.053755,-0.500145,-0.886329,-0.713294,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,HEEH,Rocklin,False,0.110093
9,Rocklin,EHEE_rd1_0571,0.594159,-0.927264,1.411274,0.496901,0.690020,-0.568958,1.231189,0.969395,...,-0.026760,-0.025026,-0.023442,-0.021987,-0.020647,False,EHEE,Rocklin,False,0.054651


In [30]:
list1 = [custom_run_data,loo_run_data_eehee,loo_run_data_ehee,loo_run_data_heeh,loo_run_data_hhh]
list2 = ["custom_run_data","loo_run_data_eehee","loo_run_data_ehee","loo_run_data_heeh","loo_run_data_hhh"]

for i in range(len(list1)):
    print('\t',list2[i].upper())
    diagnose_results(list1[i],actual_values_col='stabilityscore',predicted_values_col='stabilityscore_predictions',pred_prob='stabilityscore_prob_predictions')

	 CUSTOM_RUN_DATA
	UNSPECIFIED TOPOLOGY RESULTS:

TP count: 401
TN count: 2270
FP count: 381
FN count: 180
Accuracy: 0.8264

most sure the prediction was in the times it was correct that it was stable: 0.98 
	avg. value: 0.78
most sure the prediction was in the times it was correct that it was unstable: 0.98 
	avg. value: 0.15
most sure the prediction was in the times it was incorrect about it being stable: 0.98 
	avg. value: 0.70
most sure the prediction was in the times it was incorrect about it being unstable: 0.98 
	avg. value: 0.28

	EEHEE TOPOLOGY RESULTS:
TP count: 96
TN count: 779
FP count: 92
FN count: 83

most sure the prediction was in the times it was correct that it was stable:
 0.8945245257383657
most sure the prediction was in the times it was correct that it was unstable:
 0.4950444382488708
most sure the prediction was in the times it was incorrect about it being stable:
 0.8827408798261747
most sure the prediction was in the times it was incorrect about it being unsta

## Topology-specific models
* most probably will overfit? 
    * but if I want to predict "HHH", all other topologies might be serving as noise
* test: make a simple model for this, then use it on larger data and see how it compares